# Browser Automation Homework ROUND 2
Due 7-16<br>
Completed by: **Betül Sarikaya**

We're going to the United States trademark data base and going to collect all of Nike (or any company's) active trademarks.

We only want the live trademarks, and we want the `serial` number and the link to an image of each trademark. Feel free to collect the `wordmark` if you like.


Note: if you get asked if you're a bot, just complete the challenges manually.

In [43]:
import os
import random
import time

from playwright.async_api import async_playwright, expect, Keyboard

In [44]:
os.makedirs('data/', exist_ok=True)

In [45]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36'

In [46]:
async def open_browser(headless=False, user_agent=user_agent):
    """
    Starts the automated browser and opens a new window
    """
    # Start playwright
    playwright = await async_playwright().start()

    # Open firefox browser, can use chromium (chrome) or others
    browser = await playwright.chromium.launch(headless=headless)

    # set a user agent
    context = await browser.new_context(user_agent=user_agent)
  
    # Create a new browser window
    page = await browser.new_page()

    return browser, page

In [47]:
driver, page = await open_browser()

In [48]:
url = 'https://tmsearch.uspto.gov/search/'
await page.goto(url)

<Response url='https://tmsearch.uspto.gov/search/' request=<Request url='https://tmsearch.uspto.gov/search/' method='GET'>>

Only search by the "Owner" of each trademark

In [49]:
xpath_search_options = '//*[@id="searchbar"]'
filter = page.locator(xpath_search_options)

In [50]:
await filter.click()

In [51]:
xpath_option = '//*[@id="searchbar"]'

In [52]:
await page.locator(xpath_option).click()

Find the search bar and input the search

In [53]:
xpath_searchbar = '//*[@id="searchbar"]'

In [54]:
search = page.locator(xpath_searchbar)

In [55]:
# Change this if you like
company = 'Nike'

In [56]:
await search.fill(company)

Make the search

In [57]:
await page.keyboard.press("Enter")

Filter to only live contracts
<br>In other words, filter out the dead ones.

In [58]:
xpath_filter = '//*[@id="statusDead"]'

In [59]:
dead_button = page.locator(xpath_filter)

In [60]:
# click it
await dead_button.click()

# Save results

In [61]:
# how to save what the emulator sees
source = await page.content()
with open(f'data/trademarks_{company}.html', 'w') as f:
    f.write(source)

GO TO THE NEXT PAGE AND REPEAT

In [63]:
# this is the next page button
xpath_next = "//i[@class='material-icons' and text()='navigate_next']"

In [42]:
xpath_next

'/html/body/app-root/div/app-search/app-search-results/div/div/main/div[2]/nav/ul[2]/li[4]/a/i'

In [64]:
# What is the xpath for a result, make sure this is visible before going to next page.
xpath_result = "//div[contains(@class, 'card') and contains(@class, 'result-card')]"

In [65]:
next_button = page.locator(xpath_next)

In [72]:
await expect(next_button).to_be_visible()

In [73]:
await next_button.click()

In [ ]:
#I did it with help of ChatGPT

i = 1
collect = True

while collect:
    try:
        await page.wait_for_selector(xpath_next, timeout=100)
        next_button_visible = True
    except:
        next_button_visible = False

    if next_button_visible:
        next_button = page.locator(xpath_next)
        await next_button.click()

        # Warten auf das erste sichtbare Ergebnis-Element
        await page.locator(xpath_result).first.wait_for(state='visible', timeout=5000)

        # HTML speichern
        source = await page.content()
        with open(f'data/trademarks_{company}_{i}.html', 'w', encoding='utf-8') as f:
            f.write(source)
        i += 1
    else:
        collect = False


## Parse the contents

In [ ]:
import glob
import pandas as pd
from lxml import etree, html

In [ ]:
# list the scraped pages to pars them
files = glob.glob('data/trademarks*.html')

In [77]:
# A trademark entry
xpath_trademark_cards = './/div[contains(@class, "card") and contains(@class, "result-card")]'

# the serial number
xpath_serial = './/div[div[text()[contains(., "Serial")]]]/div[2]/span'

In [81]:
import os
from lxml import html


files = [os.path.join("data", f) for f in os.listdir("data") if f.endswith(".html")]

# XPath
xpath_trademark_cards = './/div[contains(@class, "card") and contains(@class, "result-card")]'
xpath_serial = './/div[div[text()[contains(., "Serial")]]]/div[2]/span'

data = []

for fn in files:
    with open(fn, encoding='utf-8') as f:
        tree = html.fromstring(f.read())
        trademarks = tree.xpath(xpath_trademark_cards)  # <--- xpath statt findall

        for trademark in trademarks:
            serial_elem = trademark.xpath(xpath_serial)
            serial = serial_elem[0].text.strip() if serial_elem else None

            img_elem = trademark.xpath('.//img')
            img_url = img_elem[0].get('src') if img_elem else None

            row = {'serial': serial, 'img_url': img_url}
            data.append(row)

print(data)


[{'serial': '87914270', 'img_url': 'https://tmcms-docs.uspto.gov/cases/87914270/mark/large.png'}, {'serial': '87933320', 'img_url': 'https://tmcms-docs.uspto.gov/cases/87933320/mark/large.png'}, {'serial': '88179198', 'img_url': 'https://tmcms-docs.uspto.gov/cases/88179198/mark/large.png'}, {'serial': '88133042', 'img_url': 'https://tmcms-docs.uspto.gov/cases/88133042/mark/large.png'}, {'serial': '88223286', 'img_url': 'https://tmcms-docs.uspto.gov/cases/88223286/mark/large.png'}, {'serial': '88273647', 'img_url': 'https://tmcms-docs.uspto.gov/cases/88273647/mark/large.png'}, {'serial': '88285028', 'img_url': 'https://tmcms-docs.uspto.gov/cases/88285028/mark/large.png'}, {'serial': '88321344', 'img_url': 'https://tmcms-docs.uspto.gov/cases/88321344/mark/large.png'}, {'serial': '86886452', 'img_url': 'https://tmcms-docs.uspto.gov/cases/86886452/mark/large.png'}, {'serial': '86948194', 'img_url': 'https://tmcms-docs.uspto.gov/cases/86948194/mark/large.png'}, {'serial': '86891919', 'img_u

In [83]:
import pandas as pd

df = pd.DataFrame(data)
df.head(5)

,serial,img_url
0,87914270,https://tmcms-docs.uspto.gov/cases/87914270/ma...
1,87933320,https://tmcms-docs.uspto.gov/cases/87933320/ma...
2,88179198,https://tmcms-docs.uspto.gov/cases/88179198/ma...
3,88133042,https://tmcms-docs.uspto.gov/cases/88133042/ma...
4,88223286,https://tmcms-docs.uspto.gov/cases/88223286/ma...


In [84]:
df.to_csv('data/trademarks.csv', index=False)